Importing the data from the file:

In [1]:
from numpy import *
import csv
from scipy.optimize import fmin_bfgs,minimize

#data specific numbers
class1start = 0
class2start = 59
class3start = 130

#global parameters
regParam = 0.03 #use in cost function
nSPerClass = 40 #number of samples per class
nS = 3*nSPerClass

Stuff from Assignment \#1 which we will need:

In [2]:
#LReLU, optional to leave as ReLU by passing esp=0 
def lrelu(x,b=1,esp=0.01):
    y=[]
    for xi in x:
        if xi<0:
            y.append(esp*xi)
        else:
            y.append(b*xi)
    return array(y)
def lrelu_prime(x,b=1,esp=0.01):
    y=[]
    for xi in x:
        if xi<0:
            y.append(esp)
        else:
            y.append(b)
    return array(y)
#x is input vector, W is weight estimation (use random entries between 0 and 1 for first estimate, call N somewhere)
def feed_forward(x,W):
    W0 = array(W[:k*m]).reshape((k,m))
    b0 = W[k*m:k*m+k]
    W1 = array(W[(m+1)*k:(m+1)*k+n*k]).reshape((n,k))
    b1 = W[(m+1)*k+n*k:]
    z1 = dot(W0,x)+b0
    z2 = dot(W1,a1(z1))+b1
    return a2(z2)

Begin Assignment \#2 here. Much is borrowed from your code but used my own terms as I wrote it out to make sure I understood. I am going to use the binary encoding rather than one-hot so I need the ReLU and Logistic activation functions, and I'm using an ensemble cost function rather than the other method.

In [3]:
#-------------------------------The rest of the functions we should need are in here----------------------------------
#many of which have been hybrid-ed with Dr. COo
#accepts a column vector argument, returns mean and l2 norm of vector
def normalize(col): 
    l2norm = sqrt(sum((col-mean(col))*(col-mean(col)))/len(col))
    return mean(col),l2norm

#accepts no arguments, returns array of mean values for data & array of norm values
def normalize_cols(inMat):
    means=[]
    norms=[]
    for i in range(m):
        mn,nrm=normalize(inMat[:,i])
        means.append(mn)
        norms.append(nrm)
    meanValues=array(means)
    normValues=array(norms)
    return meanValues,normValues

#accepts a matrix, returns the normalized matrix
def normalizeByTrain(inMat):
    meanVals,normVals = normalize_cols(inMat)
    normedMat=inMat*0.0
    for k in range(len(inMat[:,0])):
        normedMat[k]=(inMat[k]-meanVals)/normVals
    return normedMat

#accepts input matrices X and Y and vector W
def ensemble_cost(W,X,Y):
    C = 0.0
    for i in range(len(X[0,:])):
        x=X[i,:]
        y=Y[i,:]
        C += 0.5*(dot(feed_forward(x,W)-y,feed_forward(x,W)-y)+regParam*0.5*linalg.norm(W)**2)
    return C

#Dr. Cooper's logistic function & it's derivative below + his other functions b/c I think they'll work more reliably
def S(X):
    X = X*(abs(X)<10)+10.*(X>=10)-10.*(X<=-10)
    return 1./(1+exp(-X))

def SPrime(X):
    X = X*(abs(X)<10)+10.*(X>=10)-10.*(X<=-10)
    ex = exp(-X)
    return ex/((1.+ex)*(1.+ex))

def relu(x,epsilon=0.0):
    return x*(x>0)+epsilon*x*(x<=0)

def reluPrime(x,epsilon=0.0):
    return (x>0)+epsilon*(x<=0)

def selu(x,lamda=1,alpha=1):
    return lamda*( x*(x>0)+alpha*(exp(x)-1)*(x<=0))

def seluPrime(x,lamda=1,alpha=1):
    return lamda*((x>0)+alpha*exp(x)*(x<=0))

#accepts input vector x, output vector out, and big W vector, returns a (regularized) gradient for one row
def gradient(x,out,W):#<----borrowed
    W0 = array(W[:nW0]).reshape((k,m))
    W1 = array(W[nW0+k:-n]).reshape((n,k))
    grad = W*0.0
    # Get a(z_2)
    z1 = dot(W0,x)+W[nW0:nW0+k]
    z2 = dot(W1,a1(z1))+W[-n:]
    forward = a2(z2)
    err = forward-out
    # gradient for b_1 bias weights
    finalLayerDeriv = a2p(z2)*err
    grad[-n:] = finalLayerDeriv+0.
    # gradient for W_1 weights
    grad[(nW0+k):-n] = outer(finalLayerDeriv,a1(z1)).flatten()
    # gradient for b_0 bias weights
    firstLayerDeriv = finalLayerDeriv.dot(W1)*a1p(z1)
    grad[nW0:(nW0+k)] = firstLayerDeriv+0.
    # gradient for W_0 weights
    grad[:nW0] = outer(firstLayerDeriv,x).flatten()
    return grad+regParam*W

#accepts vector W, input matrix X and output matrix Y
def gradSum(W,X,Y):
    grad = zeros(nVar)
    for i in range(len(X[:,0])): #X[:,0] is a slice of all the rows in the first column
        grad += gradient(X[i,:],Y[i,:],W)
    return grad

The code which will actually produce the neural network structure now is below:

In [4]:
file = csv.reader(open("uciWineLocation.csv","r"))
rowIndex = 0
inData = []
outData = [] #will contain truth values for each of the three classes, i.e. 1 == [true false false], used for one-hot?
outSData = [] #will contain the numerical values corresponding to each class
testBin = 3*[0]

for row in file:
    for ri in range(len(row)):
        row[ri] = float(row[ri])
    inData.append(row[1:])
    for i in range(1,4):
        #testBin[i-1] = (row[0]==float(i))#this line writes it as one-hot
        testBin[i-1] = () #this line writes it as binary 
    outData.append(testBin.copy())#takes the encoded outdata to actually run through the 
    outSData.append(row[0]) #what to compare to later, the numbers 1,2,3
#Peeling off the training data into vectors
trainX = array(inData[:nSPerClass]+inData[class2start:class2start+nSPerClass]+inData[class3start:class3start+nSPerClass]) #rows are input data
trainY = array(outData[:nSPerClass]+outData[class2start:class2start+nSPerClass]+outData[class3start:class3start+nSPerClass]).astype(float) #rows are output data
trainSY = array(outSData[:nSPerClass]+outSData[class2start:class2start+nSPerClass]+outSData[class3start:class3start+nSPerClass]).astype(float) #thing to check results with
m = len(trainX[0]) #m is the number of inputs, i.e. number of columns in first row
n = 3 #n is the number of outputs desired
k = int(2*m/3+n) #k is the number of hidden layers
W = random.rand((m + 1)*k+(k + 1)*n) #our first estimate of our W
print('Number of samples: {}'.format(len(inData)))
print('Number of data vectors: {}'.format(nS))
print('Number of data items: {}'.format(m))
print('Number of hidden cells: {}'.format(k))
print(trainX)
nW0 = m*k
nW1 = k*n
nVar = nW0+nW1+k+n

#normalize the vector
trainX = normalizeByTrain(trainX)
#compute cost, choosing a1 to be relu and a2 to be logistic
a1 = lambda x: lrelu(x)
a1p = lambda x: lrelu_prime(x)
a2 = S
a2p = SPrime
#call training function, I'm going to use bfgs_min to solve just because it seems best
newW = W
for i in range(3):
    newW = fmin_bfgs(ensemble_cost,newW,fprime=gradSum,args=(trainX,trainY))


Number of samples: 178
Number of data vectors: 120
Number of data items: 13
Number of hidden cells: 11
[[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.282e+01 3.370e+00 2.300e+00 ... 7.200e-01 1.750e+00 6.850e+02]
 [1.358e+01 2.580e+00 2.690e+00 ... 7.400e-01 1.800e+00 7.500e+02]
 [1.340e+01 4.600e+00 2.860e+00 ... 6.700e-01 1.920e+00 6.300e+02]]
         Current function value: 1.106804
         Iterations: 33
         Function evaluations: 158
         Gradient evaluations: 146
         Current function value: 1.106804
         Iterations: 0
         Function evaluations: 74
         Gradient evaluations: 62
         Current function value: 1.106804
         Iterations: 0
         Function evaluations: 74
         Gradient evaluations: 62


In [5]:
#the test data
testX = array(inData[nSPerClass:class2start]+inData[class2start+nSPerClass:class3start]+inData[class3start+nSPerClass:]) #rows are the rest of the input data
testY = array(outData[nSPerClass:class2start]+outData[class2start+nSPerClass:class3start]+outData[class3start+nSPerClass:]).astype(float) #rows are rest of output data
testSY = array(outSData[nSPerClass:class2start]+outSData[class2start+nSPerClass:class3start]+outSData[class3start+nSPerClass:]).astype(float) #thing to check results with    
binTest = 
testX = normalizeByTrain(testX)
#now feed forward again but on test data with theoretically more accurate W
pred=[]
for i in arange(len(testX[:,0])): #i iterates over number of rows in training data
    x=testX[i,:] #x grabs the i-th row
    result=feed_forward(x,newW)
    pred.append(result)

print(array(pred))
print((array(pred)+0.5).astype(int))

[[0.80939128 0.18656869 0.17530175]
 [0.65460699 0.26358702 0.29845248]
 [0.91495864 0.08701188 0.09697915]
 [0.56258268 0.27345968 0.37792653]
 [0.71496832 0.27494569 0.21656329]
 [0.77803318 0.12087301 0.22841051]
 [0.88841764 0.11201389 0.11856756]
 [0.83678222 0.16116658 0.15628108]
 [0.79565293 0.1967     0.18733489]
 [0.87358694 0.12569529 0.13039767]
 [0.83934054 0.15927141 0.15449574]
 [0.89617832 0.10513751 0.111984  ]
 [0.92074196 0.0819204  0.09163942]
 [0.89031855 0.10880022 0.11859584]
 [0.82174751 0.17360181 0.1682023 ]
 [0.79746972 0.19648413 0.18431558]
 [0.85686117 0.14166585 0.14229268]
 [0.86689247 0.13117344 0.13616713]
 [0.89194619 0.1089183  0.11557424]
 [0.16900111 0.82930829 0.14355574]
 [0.16604188 0.82860664 0.17576412]
 [0.1847707  0.80012744 0.20082077]
 [0.27612288 0.62417712 0.33950354]
 [0.1393643  0.84950617 0.16361465]
 [0.20831198 0.7813655  0.21099208]
 [0.13923103 0.64609902 0.36223605]
 [0.20974979 0.7781577  0.21512367]
 [0.14195666 0.5525652  0.44